## Stroz Friedberg Lateral Movement Triage

### A triage review of Lateral Movement activity

In [ ]:
# Standard imports and variable definitions

import pandas
import os
from pyvelociraptor import velo_pandas
from matplotlib import pyplot as plt
import networkx as nx

# Import API config file
os.environ['VELOCIRAPTOR_API_FILE'] = "path/to/api.config.yaml"

## Stroz Friedberg Lateral Movement Hunt Jobs

### The information is gathered from Stroz Friedberg's Custom.Windows.LateralMovement hunt 

In [ ]:
# Define the Lateral Movement hunt
targetArtifact = "Custom.Windows.LateralMovement"

# Store hunt IDs
HUNT_IDS = pandas.DataFrame(velo_pandas.DataFrameQuery("""
    SELECT hunt_id FROM hunts() WHERE artifacts =~ targetArtifact
""", targetArtifact=targetArtifact))

HUNT_IDS.set_index('hunt_id', inplace=True)
HUNT_IDS = HUNT_IDS.index.values.tolist()

# Read out hunt task summary
pandas.DataFrame(velo_pandas.DataFrameQuery("""
    SELECT hunt_id,hunt_description,timestamp(epoch=start_time) as start_time, stats.total_clients_scheduled as scheduled_clients, stats.total_clients_with_results as systems_with_results FROM hunts() WHERE artifacts =~ targetArtifact
""", targetArtifact=targetArtifact))

### Retrieve Lateral Movement hunt results

In [ ]:
# Loop through applicable hunts pull down data
huntResults = []
ArtifactHuntPairs = []

print(HUNT_IDS)

for i in HUNT_IDS:
    query = pandas.DataFrame(velo_pandas.DataFrameQuery("""
        SELECT * FROM flatten(query={SELECT hunt_info(hunt_id=HuntId).artifact_sources AS sourcetopull
                                      FROM scope()})
    """, HuntId=i, timeout=9999999))
    query.set_index('sourcetopull', inplace=True)
    query = query.index.values.tolist()
    ArtifactHuntPairs.append([i, query])
print(ArtifactHuntPairs)

for i in ArtifactHuntPairs:
    for j in i[1]:
        query = pandas.DataFrame(velo_pandas.DataFrameQuery("""
            SELECT *
            FROM hunt_results(hunt_id=HuntId, artifact=artifact)
        """, HuntId=i[0], artifact=j, timeout=999999))
        huntResults.append(query)
        print(query)

huntResults = pandas.concat(huntResults)

print(huntResults)

## Lateral Movement Connection Statistics

Outgoing Lateral Movement connection information collected to better understand typical Lateral Movement behavior in the environment, and highlight potential outliers for additional review.  

### Lateral Movement Top Originators

Systems which detailed larger than normal outgoing LM connections in the environment. 

In [ ]:
huntResults.loc[huntResults['SourceSystem'] != "-"].value_counts(subset=['SourceSystem'], sort=True ).nlargest(25).to_frame("count")

### Outgoing Lateral Movement Top Talker Accounts

Accounts attempting initiating the largest count of interactive Lateral Movement sessions

In [ ]:
huntResults.value_counts(subset=['UserAccount'], sort=True, ).nlargest(25).to_frame("count")

#### Top Lateral MovementTalkers Over Time

Visualizes top account usage over time

In [ ]:
huntResults_fixed = huntResults.copy()

# Convert EventTime to datetime if needed
if not pandas.api.types.is_datetime64_any_dtype(huntResults_fixed['EventTime']):
    try:
        # Try Unix timestamp first
        huntResults_fixed['EventTime'] = pandas.to_datetime(huntResults_fixed['EventTime'], unit='s', errors='ignore')
        if not pandas.api.types.is_datetime64_any_dtype(huntResults_fixed['EventTime']):
            # Try string parsing
            huntResults_fixed['EventTime'] = pandas.to_datetime(huntResults_fixed['EventTime'], errors='coerce')
    except:
        huntResults_fixed['EventTime'] = pandas.to_datetime(huntResults_fixed['EventTime'], errors='coerce')

# Remove invalid datetime entries
huntResults_fixed = huntResults_fixed.dropna(subset=['EventTime'])

# Remove invalid datetime entries
huntResults_fixed = huntResults_fixed.dropna(subset=['EventTime'])
talkerData = pandas.DataFrame({
    'Date' : huntResults_fixed['EventTime'].dt.strftime('%Y-%m-%d'),
    'User' : huntResults_fixed['UserAccount']
})

# Add a Count column 
results = talkerData.groupby([talkerData['Date'], talkerData['User']]).size().to_frame('count').reset_index()

# Figure out the average
mean = results['count'].mean()
top = results['count'].quantile(0.98)

# Create dataset that ONLY returns login counts ABOVE the typical average
resultsMean = results[results['count'] > mean]

# Create scatter plot 
fig, ax = plt.subplots()
ax.scatter(resultsMean['Date'], resultsMean['count'])

# Annotate only the top percent entered above in "top" variable
for k, v in resultsMean.iterrows():
    if (v['count'] > top):
        ax.annotate(v['User'], xy=(v['Date'] ,v['count']))

plt.show()


### List the accounts highlighted above

In [ ]:
results[results['count'] > top]

### Lateral Movement Outgoing Pivots

Details system connections and their relationships to potentially identify pivot points. 

In [ ]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

output_notebook(hide_banner=True)
network_df = pandas.DataFrame({
    'SourceSystem' : huntResults['SourceSystem'],
    'DestinationSystem' : huntResults['DestinationSystem'],
})

# Add a Count column 
network_df = network_df.groupby([network_df['SourceSystem'], network_df['DestinationSystem']]).size().to_frame('Weight').reset_index()

# Throw out logs with no destinations
network_df = network_df[network_df.DestinationSystem != '-']
network_df = network_df[network_df.SourceSystem != '-']

G = nx.from_pandas_edgelist(network_df, 'SourceSystem', 'DestinationSystem', True)
G.edges(data=True)

degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)

number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

title = 'LM Outgoing Connections'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Source", "@index"),
        ("Degree", "@degree")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

plot.sizing_mode = "scale_width"
#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(radius=0.5,fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'Source': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='Source', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
